Retrieve experiment results

In [30]:
import json
from os.path import join
import os

import pandas as pd

from cogspaces.pipeline import get_output_dir

from json import JSONDecodeError

basedir_ids = [43]
basedirs = [join(get_output_dir(), 'predict_multi', str(_id), 'run') for _id in basedir_ids]
res_list = []
for basedir in basedirs:
    for exp_dir in os.listdir(basedir):
        exp_dir = join(basedir, exp_dir)
        try:
            config = json.load(open(join(exp_dir, 'config.json'), 'r'))
            info = json.load(open(join(exp_dir, 'info.json'), 'r'))
        except (JSONDecodeError, FileNotFoundError):
            continue
        datasets = config['datasets']
        datasets = '__'.join(datasets)
        alpha = config['alpha']
        beta = config['beta']
        seed = config['seed']
        model = config['model']
        source = config['source']
        score = info['score']
        rank = info['rank']
        scaled = config['rescale_weights']
        res = {'datasets': datasets, 'alpha': alpha, 'beta': beta, 'seed': seed, 'rank': rank,
               'source': source, 'scaled': scaled,
              'model': model}
        for key, value in score.items():
            res[key] = value
        if basedir == join(get_output_dir(), 'predict_multi', '11', 'run') and model == 'logistic':
            continue
        res_list.append(res)
res = pd.DataFrame(res_list)

Compare results with trace norm, with and without transfer

In [31]:
res

,alpha,beta,datasets,model,rank,scaled,seed,source,test_amalric2012mathematicians,test_brainomics,...,train_gauthier2009resonance,train_gauthier2010resonance,train_hcp,train_henson2010faces,train_knops2009recruitment,train_mean,train_pinel2007fast,train_pinel2009twins,train_pinel2012archi,train_vagharchakian2012temporal
0,0.000000,0.0001,brainpedia__hcp__brainomics,logistic,9,False,1230275727,hcp_rs_positive_single,0.903704,0.881797,...,1.000000,1.000000,0.958100,1.000000,1.000000,0.966571,0.983333,1.000000,0.994555,1.0000
1,0.000316,0.0000,brainpedia__hcp__brainomics,trace,56,True,1230275727,hcp_rs_positive_single,0.859259,0.894799,...,0.950000,0.867816,0.949240,1.000000,0.858974,0.944255,0.939394,0.992788,0.931942,1.0000
2,0.000000,0.1000,brainpedia__hcp__brainomics,logistic,9,False,1864370243,hcp_rs_positive_single,0.488889,0.728132,...,0.142857,0.155172,0.538608,0.875000,0.162393,0.539354,0.715152,0.711538,0.524501,0.5875
3,0.000100,0.0000,brainpedia__hcp__brainomics,trace,82,True,1230275727,hcp_rs_positive_single,0.877778,0.895981,...,1.000000,1.000000,0.983080,1.000000,1.000000,0.984925,0.981818,1.000000,0.994555,1.0000
4,0.000000,0.0010,brainpedia__hcp__brainomics,logistic,9,False,1864370243,hcp_rs_positive_single,0.885185,0.904255,...,1.000000,0.959770,0.908755,1.000000,0.854701,0.918150,0.951515,1.000000,0.876588,1.0000
5,0.000032,0.0000,brainpedia__hcp__brainomics,trace,375,True,1230275727,hcp_rs_positive_single,0.859259,0.836879,...,1.000000,1.000000,0.991632,1.000000,1.000000,0.991073,0.980303,1.000000,0.993648,1.0000
6,0.000010,0.0000,brainpedia__hcp__brainomics,trace,377,True,1230275727,hcp_rs_positive_single,0.866667,0.842790,...,1.000000,1.000000,0.991755,1.000000,1.000000,0.991210,0.980303,1.000000,0.995463,1.0000
7,0.000100,0.0000,brainpedia__hcp__brainomics,trace,83,False,1230275727,hcp_rs_positive_single,0.870370,0.888889,...,1.000000,1.000000,0.909924,1.000000,1.000000,0.928679,0.959091,1.000000,0.966425,1.0000
8,0.000000,0.0001,brainpedia__hcp__brainomics,logistic,9,False,1864370243,hcp_rs_positive_single,0.881481,0.907801,...,1.000000,1.000000,0.959823,1.000000,1.000000,0.967906,0.987879,1.000000,0.992740,1.0000
9,0.000032,0.0000,brainpedia__hcp__brainomics,trace,158,False,1230275727,hcp_rs_positive_single,0.877778,0.877069,...,1.000000,1.000000,0.953116,1.000000,1.000000,0.963656,1.000000,1.000000,1.000000,1.0000


In [34]:
df_agg = res.groupby(by=['scaled', 'source', 'model', 'datasets', 'alpha', 'beta']).aggregate(['mean', 'std'])

df_agg = df_agg.fillna(0)


dfs = {}
for dataset in ['brainpedia']:
    this_df = df_agg.query("datasets in ['%s__hcp', '%s']" % (dataset, dataset))
    indices = this_df[('test_mean', 'mean')].groupby(level=['scaled', 'source', 'model', 'datasets']).aggregate('idxmax')
    this_df = this_df.loc[indices.values]
    dfs[dataset] = this_df

In [43]:
df_agg[test_name]

test_amalric2012mathematicians  \
                                                                                                             mean   
scaled source                 model    datasets                    alpha    beta                                    
False  hcp_rs_positive_single logistic brainpedia__hcp__brainomics 0.000000 0.0001                       0.892593   
                                                                            0.0010                       0.885185   
                                                                            0.0100                       0.692593   
                                                                            0.1000                       0.488889   
                              trace    brainpedia__hcp__brainomics 0.000032 0.0000                       0.877778   
                                                                   0.000100 0.0000                       0.870370   
                                                                   0.000316 0.0000                       0.840741   
                                                                   0.001000 0.0000                       0.803704   
                                                                   0.003162 0.0000                       0.548148   
True   hcp_rs_positive_single trace    brainpedia__hcp__brainomics 0.000000 0.0000                       0.896296   
                                                                   0.000010 0.0000                       0.866667   
                                                                   0.000032 0.0000                       0.859259   
                                                                   0.000100 0.0000                       0.877778   
                                                                   0.000316 0.0000                       0.859259   
                                                                   0.001000 0.0000                       0.840741   
                                                                   0.003162 0.0000                       0.651852   

                                                                                   test_cauvet2009muslang  \
                                                                                                     mean   
scaled source                 model    datasets                    alpha    beta                            
False  hcp_rs_positive_single logistic brainpedia__hcp__brainomics 0.000000 0.0001               0.358796   
                                                                            0.0010               0.314815   
                                                                            0.0100               0.296296   
                                                                            0.1000               0.171296   
                              trace    brainpedia__hcp__brainomics 0.000032 0.0000               0.370370   
                                                                   0.000100 0.0000               0.379630   
                                                                   0.000316 0.0000               0.388889   
                                                                   0.001000 0.0000               0.375000   
                                                                   0.003162 0.0000               0.300926   
True   hcp_rs_positive_single trace    brainpedia__hcp__brainomics 0.000000 0.0000               0.370370   
                                                                   0.000010 0.0000               0.375000   
                                                                   0.000032 0.0000               0.370370   
                                                                   0.000100 0.0000               0.370370   
                                                                   0.000316 0.0000               0.375000   
      

Compare results with 3 datasets

In [26]:
result_df = df_agg

In [27]:
names = result_df.columns.levels[0].values
test_name = []
train_name = []
for name in names:
    if name.startswith('test'):
        test_name.append((name, 'mean'))
    else:
        train_name.append((name, 'mean'))

In [28]:
result_df[['test_mean', 'train_mean', 'rank']]

test_mean  \
                                                                            mean   
normalized source                 model    datasets   alpha    beta                
False      hcp_rs_positive_single logistic brainpedia 0.000000 0.00000  0.608354   
                                                               0.00001  0.620102   
                                                               0.00010  0.625250   
                                                               0.00100  0.614297   
                                                               0.01000  0.523297   
                                                               0.10000  0.416787   
                                  trace    brainpedia 0.000000 0.00000  0.617789   
                                                      0.000001 0.00000  0.617724   
                                                      0.000003 0.00000  0.619440   
                                                      0.000010 0.00000  0.620164   
                                                      0.000032 0.00000  0.618978   
                                                      0.000100 0.00000  0.619440   
                                                      0.000316 0.00000  0.615090   
                                                      0.001000 0.00000  0.575694   

                                                                                  \
                                                                             std   
normalized source                 model    datasets   alpha    beta                
False      hcp_rs_positive_single logistic brainpedia 0.000000 0.00000  0.003295   
                                                               0.00001  0.001282   
                                                               0.00010  0.003475   
                                                               0.00100  0.006434   
                                                               0.01000  0.005781   
                                                               0.10000  0.005278   
                                  trace    brainpedia 0.000000 0.00000  0.003199   
                                                      0.000001 0.00000  0.002325   
                                                      0.000003 0.00000  0.001346   
                                                      0.000010 0.00000  0.003214   
                                                      0.000032 0.00000  0.002431   
                                                      0.000100 0.00000  0.002391   
                                                      0.000316 0.00000  0.004682   
                                                      0.001000 0.00000  0.004386   

                                                                       train_mean  \
                                                                             mean   
normalized source                 model    datasets   alpha    beta                 
False      hcp_rs_positive_single logistic brainpedia 0.000000 0.00000   0.992603   
                                                               0.00001   0.992603   
                                                               0.00010   0.988973   
                                                               0.00100   0.935139   
                                                               0.01000   0.703151   
                                                               0.10000   0.507668   
                                  trace    brainpedia 0.000000 0.00000   0.992603   
                                                      0.000001 0.00000   0.992603   
                                                      0.000003 0.00000   0.992603   
                                                      0.000010 0.00000   0.992603   
                                                      0.000032 0.00000   0.992466   
                       

In [74]:
df_trace = df_agg.query("model == 'trace'")[test_name]
alphas = df_trace.index.get_level_values('alpha').values

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

In [ ]:

fig = plt.figure()
ax = fig.add_subplot(111)
for name, data in df_trace.items():
    data = data - data.iloc[0]
    ax.plot(data.index.get_level_values('alpha'), data, label=name[0])
ax.set_xscale('log')
ax.set_ylim([-0.1, 0.1])

In [ ]:
import seaborn as sns

sns.plot(data=df_trace)

In [57]:
df_agg.query("model == 'logistic'")[test_name]

test_amalric2012mathematicians  \
                                                                                                        mean   
normalized source                 model    datasets        alpha beta                                          
False      hcp_rs_positive_single logistic brainpedia__hcp 0.0   0.000000e+00                       0.856852   
                                                                 1.000000e-10                       0.856852   
                                                                 1.000000e-09                       0.856852   
                                                                 1.000000e-08                       0.856852   
                                                                 1.000000e-07                       0.857037   
                                                                 1.000000e-06                       0.857222   
                                                                 1.000000e-05                       0.857222   
                                                                 1.000000e-04                       0.855926   
                                                                 1.000000e-03                       0.832593   
                                                                 1.000000e-02                       0.677593   
                                                                 1.000000e-01                       0.521852   

                                                                              test_cauvet2009muslang  \
                                                                                                mean   
normalized source                 model    datasets        alpha beta                                  
False      hcp_rs_positive_single logistic brainpedia__hcp 0.0   0.000000e+00               0.335185   
                                                                 1.000000e-10               0.335185   
                                                                 1.000000e-09               0.335185   
                                                                 1.000000e-08               0.335185   
                                                                 1.000000e-07               0.335185   
                                                                 1.000000e-06               0.335185   
                                                                 1.000000e-05               0.335648   
                                                                 1.000000e-04               0.340278   
                                                                 1.000000e-03               0.334954   
                                                                 1.000000e-02               0.292824   
                                                                 1.000000e-01               0.173611   

                                                                              test_devauchelle2009sentence  \
                                                                                                      mean   
normalized source                 model    datasets        alpha beta                                        
False      hcp_rs_positive_single logistic brainpedia__hcp 0.0   0.000000e+00                     0.230000   
                                                                 1.000000e-10                     0.230000   
                                                                 1.000000e-09                     0.230000   
                                                                 1.000000e-08                     0.230000   
                                                                 1.000000e-07                     0.230000   
                                                                 1.000000e-06                     0.229444   
                                                              